In [ ]:
!pip install kaggle --quiet

# Upload your Kaggle API key
from google.colab import files
files.upload()  # Upload kaggle.json

# Move the kaggle.json file to the appropriate directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d toxicmender/20bn-jester -p /content/dataset --unzip


Dataset URL: https://www.kaggle.com/datasets/toxicmender/20bn-jester
License(s): unknown
100% 11.9G/11.9G [09:20<00:00, 23.9MB/s]
100% 11.9G/11.9G [09:21<00:00, 22.8MB/s]


In [ ]:
!ls /content/dataset


Test  Test.csv	Train  Train.csv  Validation  Validation.csv


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Add the 'lib' directory to Python path
import sys
sys.path.insert(0, '/content/drive/MyDrive/lib')  # Ensure correct path

# Verify if files are accessible
!ls /content/drive/MyDrive/lib


Mounted at /content/drive
data_loader.py	image.py  kaggle.json  __pycache__  resnet_model.py


In [ ]:
!pip install keras_preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
# # Import necessary libraries
# import os
# from keras.optimizers import SGD
# from keras.models import load_model
# from keras.callbacks import ModelCheckpoint
# from data_loader import DataLoader
# from image import ImageDataGenerator
# from resnet_model import Resnet3DBuilder

# # Define dataset paths
# root_directory = '/content/dataset'  # Updated path to extracted Kaggle dataset
# labels_csv_path = os.path.join(root_directory, 'labels_extracted.csv')
# train_csv_path = os.path.join(root_directory, 'train_extracted.csv')
# val_csv_path = os.path.join(root_directory, 'validation_extracted.csv')
# train_path = os.path.join(root_directory, 'Train')
# val_path = os.path.join(root_directory, 'Validation')

# # Load data using DataLoader
# data = DataLoader(labels_csv_path, train_csv_path, val_csv_path)
# Import necessary libraries
import os
from keras.optimizers import SGD
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from data_loader import DataLoader
from image import ImageDataGenerator
from resnet_model import Resnet3DBuilder

# Define dataset paths
root_directory = '/content/'  # Updated path to extracted Kaggle dataset
labels_csv_path = os.path.join(root_directory, 'drive/MyDrive/dataset/labels_extracted.csv')
train_csv_path = os.path.join(root_directory, 'drive/MyDrive/dataset/train_extracted.csv')
val_csv_path = os.path.join(root_directory, 'drive/MyDrive/dataset/validation_extracted.csv')
train_path = os.path.join(root_directory, 'dataset/Train')
val_path = os.path.join(root_directory, 'dataset/Validation')

# Load data using DataLoader
data = DataLoader(labels_csv_path, train_csv_path, val_csv_path)


In [ ]:
# Model parameters
WIDTH = 96
HEIGHT = 64
TARGET_SIZE = (HEIGHT, WIDTH)
BATCH_SIZE = 32
N_FRAMES = 16
SKIP = 1
INPUT_SHAPE = (N_FRAMES,) + TARGET_SIZE + (3,)
N_CLASSES = 10
DROP_RATE = 0.4
LEARNING_RATE = 0.05
EPOCHS_PER_SEGMENT = 2  # Train for 2 epochs per segment


In [ ]:
# Build ResNet-101 model
model = Resnet3DBuilder.build_resnet_101(
    input_shape=INPUT_SHAPE,
    num_outputs=N_CLASSES,
    drop_rate=DROP_RATE
)

# Compile the model
optimizer = SGD(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 16, 64, 96, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 8, 32, 48, 64)  │         65,920 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 8, 32, 48, 64)  │            256 │ conv3d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 8, 32, 48, 64)  │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling3d             │ (None, 4, 16, 24, 64)  │              0 │ activation[0][0]       │
│ (MaxPooling3D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 4, 16, 24, 64)  │          4,160 │ max_pooling3d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 4, 16, 24, 64)  │            256 │ conv3d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 4, 16, 24, 64)  │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (None, 4, 16, 24, 64)  │        110,656 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 4, 16, 24, 64)  │            256 │ conv3d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 4, 16, 24, 64)  │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_4 (Conv3D)         │ (None, 4, 16, 24, 256) │         16,640 │ max_pooling3d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_3 (Conv3D)         │ (None, 4, 16, 24, 256) │         16,640 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 4, 16, 24, 256) │              0 │ conv3d_4[0][0],        │
│                           │                        │                │ conv3d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 4, 16, 24, 256) │          1,024 │ add[0][0]              │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 4, 16, 24, 256) │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_5 (Conv3D)    

 Total params: 85,408,266 (325.81 MB)

 Trainable params: 85,310,602 (325.43 MB)

 Non-trainable params: 97,664 (381.50 KB)

In [ ]:
# Define the training function
def train_model(model, initial_epoch=0, total_epochs=1, model_name='resnet_101', save_to_drive=True):
    # Initialize data generator
    image_datagen = ImageDataGenerator()

    # Training generator
    train_generator = image_datagen.flow_video_from_dataframe(
        dataframe=data.train_df,
        directory=train_path,
        path_classes=labels_csv_path,
        x_col='video_id',
        y_col='label',
        target_size=TARGET_SIZE,
        class_mode='categorical',
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42,
        nb_frames=N_FRAMES,
        skip=SKIP
    )

    # Validation generator
    validation_generator = image_datagen.flow_video_from_dataframe(
        dataframe=data.val_df,
        directory=val_path,
        path_classes=labels_csv_path,
        x_col='video_id',
        y_col='label',
        target_size=TARGET_SIZE,
        class_mode='categorical',
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        shuffle=False,
        seed=42,
        nb_frames=N_FRAMES,
        skip=SKIP
    )

    # Handle missing files in generator
    def handle_missing_files(generator):
        while True:
            try:
                x, y = next(generator)
                yield x, y
            except (FileNotFoundError, ValueError) as e:
                print(f"Warning: {e}. Skipping batch.")

    train_generator = handle_missing_files(train_generator)
    validation_generator = handle_missing_files(validation_generator)

    # Create directories for saving models if they don't exist
    os.makedirs('/content/output/models', exist_ok=True)
    if save_to_drive:
        os.makedirs('/content/drive/MyDrive/output/models', exist_ok=True)

    # Save model checkpoints to both runtime and Drive
    model_checkpoint_runtime = ModelCheckpoint(
        f'/content/output/models/{model_name}_{{epoch:02d}}.keras',
        save_weights_only=False,
        save_best_only=False
    )

    model_checkpoint_drive = ModelCheckpoint(
        f'/content/drive/MyDrive/output/models/{model_name}_{{epoch:02d}}.keras',
        save_weights_only=False,
        save_best_only=False
    )

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=len(data.train_df) // BATCH_SIZE,
        validation_data=validation_generator,
        validation_steps=len(data.val_df) // BATCH_SIZE,
        epochs=total_epochs,
        initial_epoch=initial_epoch,
        verbose=1,
        callbacks=[model_checkpoint_runtime, model_checkpoint_drive]
    )

    # Save the final model to both runtime and Drive
    model.save(f'/content/output/models/{model_name}_final.keras')
    if save_to_drive:
        model.save(f'/content/drive/MyDrive/output/models/{model_name}_final.keras')

    return history


In [ ]:
# Ensure output directory exists
os.makedirs('/content/drive/MyDrive/output/models', exist_ok=True)


In [ ]:
# Train the model for the total epochs split into segments
history = train_model(model, initial_epoch=0, total_epochs=EPOCHS_PER_SEGMENT)


Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 1/2
250/250 ━━━━━━━━━━━━━━━━━━━━ 250s 734ms/step - accuracy: 0.1063 - loss: 13.1177 - val_accuracy: 0.1169 - val_loss: 12.8610
Epoch 2/2
250/250 ━━━━━━━━━━━━━━━━━━━━ 169s 675ms/step - accuracy: 0.1530 - loss: 12.8469 - val_accuracy: 0.1341 - val_loss: 13.0141


In [ ]:
# Continue Training from Saved Checkpoints
for segment in range(1, 6):  # Example: 5 segments of 2 epochs each = 10 total epochs
    checkpoint_path = f'/content/drive/MyDrive/output/models/resnet_101_{segment * EPOCHS_PER_SEGMENT:02d}.keras'
    if os.path.exists(checkpoint_path):
        # Load the saved model
        model = load_model(checkpoint_path)
        print(f"Loaded model from {checkpoint_path}")
    else:
        print(f"Checkpoint {checkpoint_path} not found. Training will start from scratch for this segment.")

    # Continue training from the loaded checkpoint
    history = train_model(
        model,
        initial_epoch=segment * EPOCHS_PER_SEGMENT,
        total_epochs=(segment + 1) * EPOCHS_PER_SEGMENT
    )


Loaded model from /content/drive/MyDrive/output/models/resnet_101_02.keras
Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 3/4
250/250 ━━━━━━━━━━━━━━━━━━━━ 221s 633ms/step - accuracy: 0.2773 - loss: 12.4207 - val_accuracy: 0.2777 - val_loss: 12.7396
Epoch 4/4
250/250 ━━━━━━━━━━━━━━━━━━━━ 154s 616ms/step - accuracy: 0.3917 - loss: 12.0057 - val_accuracy: 0.3592 - val_loss: 12.2865
Loaded model from /content/drive/MyDrive/output/models/resnet_101_04.keras
Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.
Epoch 5/6
250/250 ━━━━━━━━━━━━━━━━━━━━ 232s 687ms/step - accuracy: 0.4854 - loss: 11.7583 - val_accuracy: 0.4945 - val_loss: 11.7113
Epoch 6/6
250/250 ━━━━━━━━━━━━━━━━━━━━ 165s 660ms/step - accuracy: 0.5294 - loss: 11.5453 - val_accuracy: 0.5188 - val_loss: 11.6531
Loaded model from /content/drive/MyDrive/output/models/resnet_101_06.keras
Found 8000 video folders belonging to 10 cl

In [ ]:
from keras.models import load_model

# Load the final trained model
final_model_path = '/content/drive/MyDrive/output/models/resnet_101_final.keras'
model = load_model(final_model_path)
print(f"Loaded model from {final_model_path}")

# Prepare the validation generator
image_datagen = ImageDataGenerator()

validation_generator = image_datagen.flow_video_from_dataframe(
    dataframe=data.val_df,
    directory=val_path,
    path_classes=labels_csv_path,
    x_col='video_id',
    y_col='label',
    target_size=TARGET_SIZE,
    class_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42,
    nb_frames=N_FRAMES,
    skip=SKIP
)

# Ensure validation generator works properly
try:
    sample_batch = next(validation_generator)
    print(f"Batch shape: {sample_batch[0].shape}, Labels shape: {sample_batch[1].shape}")
except Exception as e:
    print(f"Error in validation generator: {e}")

# Helper function to wrap the generator
def wrapped_generator(generator):
    while True:
        try:
            x, y = next(generator)
            yield x, y
        except StopIteration:
            break

# Wrap the validation generator
wrapped_validation_generator = wrapped_generator(validation_generator)

# Evaluate the model
final_loss, final_accuracy = model.evaluate(
    wrapped_validation_generator,
    steps=len(data.val_df) // BATCH_SIZE,
    verbose=1
)

print(f"Final validation loss: {final_loss}")
print(f"Final validation accuracy: {final_accuracy}")



Loaded model from /content/drive/MyDrive/output/models/resnet_101_final.keras
Found 2000 video folders belonging to 10 classes.
Batch shape: (32, 16, 64, 96, 3), Labels shape: (32, 10)
62/62 ━━━━━━━━━━━━━━━━━━━━ 53s 597ms/step - accuracy: 0.6242 - loss: 11.1439
Final validation loss: 11.127492904663086
Final validation accuracy: 0.627540647983551


In [ ]:
!ls /content/drive/MyDrive/output/models/


resnet_101_01.keras  resnet_101_05.keras  resnet_101_09.keras  resnet_101_final.keras
resnet_101_02.keras  resnet_101_06.keras  resnet_101_10.keras
resnet_101_03.keras  resnet_101_07.keras  resnet_101_11.keras
resnet_101_04.keras  resnet_101_08.keras  resnet_101_12.keras
